In [ ]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.datasets as datasets
from PIL import Image
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

In [ ]:
# force to reload the internal modules
import sys
sys.modules.pop('classifier_data', None)
sys.modules.pop('cnn_classifier', None)

import classifier_data
import cnn_classifier

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

print(f"Using device: {device}")

In [ ]:
# --- Extract data from local files

data_path = os.path.join(".", "datasets", "apple2orange64")
img_size = 64

transformations = [
    transforms.Resize(int(img_size * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

train_dataset = classifier_data.BinaryClassificationImageDataset(
    data_path,
    transformations=transformations,
    mode="train"
)

test_dataset = classifier_data.BinaryClassificationImageDataset(
    data_path,
    transformations=transformations,
    mode="validation"
)

classifier_data.show_sample(train_dataset, "Apple", "Orange")

In [ ]:
# --- Create the model

model = cnn_classifier.ConvolutionalClassifier()

In [ ]:
# --- Test the model before training

test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
x_test_img, y_test = next(iter(test_data_loader))

with torch.no_grad():
    y_test_pred = model(x_test_img).argmax(dim=1)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)

In [ ]:
# --- Train the model

model.to(device)


epoch_values, loss_values = cnn_classifier.train(
    model,
    device=device,
    train_data=train_dataset,
    nb_epochs=100,
    batch_size=64,
    learning_rate=0.01
)

model.to(cpu)


plt.title("Loss over epochs")
plt.plot(epoch_values, loss_values)
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# --- Test the model after training

test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
x_test_img, y_test = next(iter(test_data_loader))

with torch.no_grad():
    y_test_pred = model(x_test_img).argmax(dim=1)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)

In [ ]:
# --- Test the model after training

test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


y_test_pred = []
y_test = []
for i, batch in enumerate(test_data_loader):
    x_test_img, y_test_ = batch
    y_test_pred.append(model(x_test_img).argmax(dim=1).item())
    y_test.append(y_test_.item())

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)

**Import Data to Classify**

In [ ]:
# input_dataset_folder = "inferences/fdcgan/"
input_dataset_folder = "inferences/cycle-gan/fdcgan/"

input_dataset = classifier_data.BinaryClassificationImageDataset(
    input_dataset_folder,
    transformations=transformations,
    mode="apple2orange64"
    # mode="orange2apple64"
)

input_dataset_loader = DataLoader(input_dataset, batch_size=1, shuffle=False)

print(f"Number of images: {len(input_dataset)}")
print(type(input_dataset_loader))
# x_test_img, y_test = next(iter(input_dataset_loader))

y_test_pred = []
y_test = []
for i, batch in enumerate(input_dataset_loader):
    x_test_img, y_test_ = batch
    y_test_pred.append(model(x_test_img).argmax(dim=1).item())
    y_test.append(y_test_.item())

# with torch.no_grad():
#     y_test_pred = model(x_test_img).argmax(dim=1)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)